In [ ]:
import { OllamaEmbeddings, Ollama } from "npm:/@langchain/ollama@0.1.5";

const model = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "deepseek-r1",
});

const embeddings = new OllamaEmbeddings({
  model: "nomic-embed-text",
  baseUrl: "http://localhost:11434/",
});

In [ ]:
import {TextLoader} from 'langchain/document_loaders/fs/text'
// 加载数据源
const loader = new TextLoader('data/qiu.txt')
const docs = await loader.load()

In [ ]:
import { RecursiveCharacterTextSplitter } from 'langchain/text_splitter';
// 分割原始数据
const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 100
})

const splitDocs = await splitter.splitDocuments(docs);

In [ ]:
import { MemoryVectorStore} from 'langchain/vectorstores/memory';
// 构建 vectorstore
const vectorstore = new MemoryVectorStore(embeddings)
await vectorstore.addDocuments(splitDocs)


In [ ]:
const retriever = vectorstore.asRetriever(2)

In [ ]:
const res = await retriever.invoke(
  "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
);


In [ ]:
res

In [ ]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const converDocsToString = (documents: Document[]) => {
  return documents.map((document) => document.pageContent).join("\n");
};

const contextRetrieverChain = RunnableSequence.from([
  (input) => input.question,
  retriever,
  converDocsToString,
]);


In [ ]:
const result = await contextRetrieverChain.invoke({
  question: "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论",
});

console.log(result);


: 

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);


In [2]:
import { StringOutputParser } from "@langchain/core/output_parsers";
import { OllamaEmbeddings, Ollama } from "npm:/@langchain/ollama@0.1.5";
import { TextLoader } from "langchain/document_loaders/fs/text";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const model = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "deepseek-r1",
});

const embeddings = new OllamaEmbeddings({
  model: "nomic-embed-text",
  baseUrl: "http://localhost:11434/",
});

// 1、加载原始数据
const loader = new TextLoader("data/qiu.txt");
const docs = await loader.load();

// 2、分割原始数据
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 500,
  chunkOverlap: 100,
});
const splitDocs = await splitter.splitDocuments(docs);

// 3、构建 vectorstore
const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

// 4、设置 检索返回的粒度
const retriever = vectorstore.asRetriever(2);

// 转换文本数据
const converDocsToString = (documents: Document[]) => {
  return documents.map((document) => document.pageContent).join("\n");
};

const contextRetrieverChain = RunnableSequence.from([
  (input) => input.question,
  retriever,
  converDocsToString,
]);

// 5、创建提示模版
const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

const ragChain = RunnableSequence.from([
  { question: (input) => input.question, context: contextRetrieverChain },
  prompt,
  model,
  new StringOutputParser(),
]);

const answer = await ragChain.invoke({
  question: "详细描述原文中有什么跟直升机相关的场景",
});

console.log(answer);


<think>
好的，用户让我根据《球状闪电》的原文详细描述与直升机相关的场景，并且强调我是终极原著党，只引用原文。首先我需要仔细回忆书中的内容，看看有没有提到直升机的部分。

记得在小说里，陈博士和同事们经常进行高空实验，尤其是用飞机携带装置到雷暴区域触发球状闪电。不过具体有没有出现直升机呢？可能没有直接描写直升机，但提到了其他飞行器。比如第二部中，他们使用了改装的无人机或者飞艇来收集数据或进行实验。

需要确认原文中的描述是否涉及直升机。根据提供的文本片段，里面讨论的是数学模型和物理定律，并没有提到具体的交通工具如直升机。所以可能用户的问题是基于某个场景，但原作并未直接出现直升机相关的内容。

接下来要检查是否有间接关联，比如在实验过程中使用的飞行器类型。例如，第三部中可能有其他载具的使用，但同样没有明确说是直升机。因此正确的回答应该是原文中没有提到直升机相关的场景，并按照用户指示回复“原文中没有相关内容”。
</think>
根据提供的文本片段和内容摘要，其中并未出现任何与**直升机（Helicopter）**相关的具体场景或描写。

所有讨论都集中在：
1.  球状闪电的数学模型构建困难性
2.  物理学本身的复杂性和基本定律在描述球状闪电时的局限性
3.  高空实验和雷暴环境下的研究工作

没有提及任何具体的飞行器类型，包括直升机、飞机或其他。文中提到的是进行实验可能用到的高空平台或装置，但并未具体说明是哪种交通工具。

因此，答案是：

原文中没有相关内容
